### 단어 수준 임베딩 예시

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [2]:
import os
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

os.chdir("/content/drive/My Drive")
data = pd.read_csv("jobkorea_all.csv")

In [3]:
from preprocess import _preprocess_company, _preprocess_qna

# 회사명, 답변 전처리
data = _preprocess_company(data)
data = _preprocess_qna(data)

In [4]:
# 회사명을 단어 사전에 추가
company_dict = pd.DataFrame({"회사명": data["회사명"].unique()})
company_dict["품사"] = "N"
company_dict.to_csv("company_dict.txt", sep="\t", index=False, header=False)

In [ ]:
!pip install konlpy

In [6]:
from konlpy.tag import Komoran

komoran = Komoran(userdic="company_dict.txt")

In [8]:
# 형태소 분리
data["답변"] = data["답변"].apply(lambda x: komoran.morphs(x))
data["답변"].head()

0    [능동, 적, 이, ㄴ, 직무, 수행, 의, 기회, 코리아세븐, 유통업, 에, 대하...
1    [내, 가족, 내, 물건, 같이, 생각, 하, 는, 책임감, 대학교, 에서, 경제학...
2    [열정, 과, 창의, 능동, 성, 을, 통하, ㄴ, 성과, 직접, 제안, 하, ㄴ,...
3    [역량, 발전, 을, 위하, ㄴ, 노력, 데이터, 분석, 능력, 소통, 능력, 코리...
4    [편안, 하, ㄴ, 서비스, 상권, 과, 시즌, 에, 맞, 는, 운영, 최근, 세븐...
Name: 답변, dtype: object

### FastText, Word2Vec

In [9]:
from gensim.models import Word2Vec, FastText

size =  16 # 단어 벡터 차원 수
window = 5 # 함께 고려할 주변 단어 개수
sg = 0 # 0이면 skipgram 방식, 1이면 cbow 방식
min_count = 1 # 일정 개수 이상 등장하는 단어에 대해서만 벡터로 만듦
alpha = 0.05 # 단어 벡터로 만들기 위한 학습 과정에 쓰이는 학습률 (learning rate)
min_alpha = 1e-4 # 학습시키며 학습률을 점차 줄여가는데, 이때 학습률의 최소값
batch_words = 128 # 학습 과정에서 한 번에 학습시킬 단어 개수

In [28]:
2 ** 16 # 16차원으로 6만개 이상의 단어를 표현할 수 있음

65536

In [10]:
ft_cb = FastText(data["답변"].to_list(), size=size, sg=sg, alpha=alpha, window=window, min_count=min_count, min_alpha=min_alpha, batch_words=batch_words)
ft_sg = FastText(data["답변"].to_list(), size=size, sg=sg, alpha=alpha, window=window, min_count=min_count, min_alpha=min_alpha, batch_words=batch_words)
wv_cb = Word2Vec(data["답변"].to_list(), size=size, sg=sg, alpha=alpha, window=window, min_count=min_count, min_alpha=min_alpha, batch_words=batch_words)
wv_sg = Word2Vec(data["답변"].to_list(), size=size, sg=sg, alpha=alpha, window=window, min_count=min_count, min_alpha=min_alpha, batch_words=batch_words)

In [13]:
ft_cb.wv.similar_by_word("고객")

[('소비자', 0.9330433011054993),
 ('하나님', 0.8482263088226318),
 ('농민', 0.8438336253166199),
 ('화주', 0.8428754806518555),
 ('소비자학', 0.8333024978637695),
 ('공직자', 0.824482798576355),
 ('조합원', 0.8228940367698669),
 ('청자', 0.8172776103019714),
 ('직원', 0.8030308485031128),
 ('상사화', 0.7985577583312988)]

In [14]:
ft_cb.wv.similar_by_word("인공지능")

[('로봇과 인공지능', 0.9466492533683777),
 ('사물 인터넷', 0.9178272485733032),
 ('사물', 0.9059036374092102),
 ('가상현실', 0.8934950828552246),
 ('인공', 0.8878297805786133),
 ('iot', 0.8845454454421997),
 ('ai', 0.8674906492233276),
 ('모바일 장치', 0.8618234395980835),
 ('블록체인', 0.8612567186355591),
 ('스마트 혁명', 0.8608694076538086)]

In [15]:
ft_sg.wv.similar_by_word("고객")

[('소비자', 0.9315643906593323),
 ('소비자학', 0.8373796343803406),
 ('청자', 0.8329489231109619),
 ('바이어', 0.8290488123893738),
 ('투자자', 0.8167580366134644),
 ('고객감동', 0.8112527132034302),
 ('농민', 0.7979772090911865),
 ('상대국', 0.7943459153175354),
 ('경쟁사', 0.7856153249740601),
 ('타인', 0.7845069169998169)]

In [16]:
ft_sg.wv.similar_by_word("인공지능")

[('로봇과 인공지능', 0.9364467859268188),
 ('사물', 0.8948230147361755),
 ('가상현실', 0.889631986618042),
 ('사물 인터넷', 0.8876920938491821),
 ('ai', 0.8847659826278687),
 ('iot', 0.8844782114028931),
 ('그리드 컴퓨팅', 0.8815866112709045),
 ('역도', 0.8640447854995728),
 ('해돋이', 0.8595154881477356),
 ('블록체인', 0.8574888706207275)]

In [17]:
wv_cb.wv.similar_by_word("고객")

[('소비자', 0.9064821004867554),
 ('농민', 0.7986710071563721),
 ('최상', 0.7978560924530029),
 ('신뢰감', 0.7850571274757385),
 ('가맹점', 0.7583389282226562),
 ('직원', 0.7481755018234253),
 ('경영진', 0.737156867980957),
 ('경쟁사', 0.7345210313796997),
 ('국민', 0.7339615821838379),
 ('바이어', 0.7307088375091553)]

In [18]:
wv_cb.wv.similar_by_word("인공지능")

[('ai', 0.8857212662696838),
 ('iot', 0.8737084865570068),
 ('사물', 0.8546755313873291),
 ('가상현실', 0.8513230085372925),
 ('이퍼', 0.8477895259857178),
 ('클라우드 컴퓨팅', 0.8406417369842529),
 ('블록체인', 0.8173487186431885),
 ('고분자', 0.8059898614883423),
 ('세탁기', 0.803713321685791),
 ('ceramic', 0.7983219623565674)]

In [19]:
wv_sg.wv.similar_by_word("고객")

[('소비자', 0.9250689744949341),
 ('직원', 0.8029738068580627),
 ('농민', 0.7976235747337341),
 ('캐슬', 0.7963955998420715),
 ('바이어', 0.7822356820106506),
 ('경영진', 0.7782505750656128),
 ('국민', 0.7772172689437866),
 ('화주', 0.7678870558738708),
 ('행원', 0.7648428678512573),
 ('최상', 0.763500988483429)]

In [20]:
wv_sg.wv.similar_by_word("인공지능")

[('ai', 0.9054152965545654),
 ('사물', 0.9005746245384216),
 ('iot', 0.8737410306930542),
 ('사물 인터넷', 0.8686914443969727),
 ('블록체인', 0.8347156047821045),
 ('가상현실', 0.8316118717193604),
 ('클라우드 컴퓨팅', 0.8256445527076721),
 ('핀테크', 0.824096143245697),
 ('고급', 0.815834105014801),
 ('금속', 0.8153983950614929)]

In [21]:
print(len(wv_sg.wv.vocab)) # 단어 벡터로 등록된 단어 개수

39463


### GloVe

In [ ]:
!pip install glove_python

In [54]:
from glove import Corpus, Glove

corpus = Corpus()
corpus.fit(data["답변"], window=5)

In [55]:
glove = Glove(no_components=16, learning_rate=0.05) # no_components: 단어 벡터 차원 수
glove.fit(corpus.matrix, epochs=5, no_threads=4, verbose=False)
glove.add_dictionary(corpus.dictionary)

In [56]:
print(glove.most_similar("고객"))
print(glove.most_similar("인공지능"))

[('상대방', 0.9812837066720906), ('제품', 0.9806804911060614), ('직무', 0.9784764523397101), ('소비자', 0.9776151369293282)]
[('핀테크', 0.9823229519517226), ('건설', 0.9801742309331771), ('패션', 0.9786412138450863), ('제조업', 0.9756950659057545)]


In [57]:
len(glove.dictionary) # 단어 벡터로 등록된 단어 개수

39463